# VizDoom Playground

In [2]:
import itertools as it
import json
import os

import cv2
import gym
from gym.spaces import Box, Discrete
import matplotlib.pyplot as plt
import numpy as np
import vizdoom as vzd

from rlpyt.envs.vizdoom.vizdoom_env import VizDoomEnv
import rlpyt.models.resnet as resnet

vzd_dir = 'vizdoom_data'
import keras
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

## Player Control

In [ ]:
game = vzd.DoomGame()
game.load_config('vizdoom_data/my_way_home_control.cfg')
game.set_doom_scenario_path('vizdoom_data/my_way_home.wad')
game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
game.set_mode(vzd.Mode.SPECTATOR)
game.set_window_visible(True)

In [ ]:
game.init()
game.new_episode()
while not game.is_episode_finished():
    s = game.get_state()
    game.advance_action()
    a = game.get_last_action()
    r = game.get_last_reward()
game.close()

## Replay Episode

In [ ]:
game = vzd.DoomGame()
game.load_config('vizdoom_data/my_way_home_control.cfg')
game.init()
game.replay_episode('../data/local/20200722/vizdoom/run_0/eval_run_0_itr_0.lmp')
while not game.is_episode_finished():
    game.advance_action()
game.close()

## Cosine Similarity Exploration

In [ ]:
np.random.seed(333)

In [3]:
env = VizDoomEnv('../experiments/vizdoom/train_SPTM.cfg', 0, [500, 1000], 270, grayscale=False, map_id='map05')

In [ ]:
positions = np.array([pos for _, pos in env.sample_states])

In [ ]:
env.plot_topdown()
plt.scatter(positions[:, 0], positions[:, 1])

In [ ]:
env.plot_topdown()
plt.scatter(env.sample_sectors[:, 0], env.sample_sectors[:, 1], c=env.sample_sectors[:, 2], s=100)

In [ ]:
start_state = env.sample_states[0]

In [ ]:
start_state_features = start_state[0].flatten() / np.linalg.norm(start_state[0])

In [ ]:
similarities = []

In [ ]:
for state, position in env.sample_states:
    state_features = state.flatten() / np.linalg.norm(state)
    similarity = np.dot(start_state_features, state_features)
    similarities.append((*position, similarity))
#     plt.figure(figsize=(7, 7))
#     plt.imshow(np.transpose(state, [1, 2, 0]))
#     plt.title('{} {}: {}'.format(*position, similarity))

In [ ]:
similarities = np.array(similarities)

In [ ]:
plt.figure(figsize=(7, 7))
env.plot_topdown()
plt.scatter(similarities[:, 0], similarities[:, 1], c=range(len(similarities)), s=100)
plt.colorbar()

## MyWayHome

In [ ]:
env = VizDoomEnv('../experiments/vizdoom/my_way_home_SPTM.cfg', 0, [1040, -300], 270, grayscale=False)

In [ ]:
env.plot_topdown()
plt.scatter(env.sample_sectors[:, 0], env.sample_sectors[:, 1], c=env.sample_sectors[:, 2], s=100)

In [ ]:
for state, position in env.sample_states:
    plt.figure(figsize=(7, 7))
    plt.imshow(np.transpose(state, [1, 2, 0]))
    plt.title('{} {}'.format(*position))

In [ ]:
obs = env.reset()

In [ ]:
env.plot_topdown()

In [ ]:
plt.imshow(env.goal_state[-1], cmap='gray')

In [ ]:
env.plot_topdown()

In [ ]:
state = env.state

In [ ]:
state.sectors[0].lines[5].y2

In [ ]:
env.goal_position

In [ ]:
env.game.get_game_variable(vzd.GameVariable.ANGLE)

In [ ]:
env.game.send_game_command("warp 200 -200")

In [ ]:
obs, reward, done, info = env.step(3)

In [ ]:
done

In [ ]:
env.state.game_variables

In [ ]:
plt.imshow(obs[-1], cmap='gray')

In [ ]:
for i in range(500):
    obs, reward, done, info = env.step(np.random.randint(3))
    if done:
        print(reward, i + 1)
        break

In [ ]:
env.plot_topdown()

In [ ]:
plt.imshow(env.visited.T, origin='lower')

## Testing SPTM Retrieval Network

In [ ]:
game = vzd.DoomGame()
game.load_config('vizdoom_data/train_SPTM_control.cfg')
game.set_doom_map('map%02d' % 5)
game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
game.set_mode(vzd.Mode.SPECTATOR)
game.set_seed(100)
game.set_window_visible(True)

In [ ]:
game.init()
game.new_episode()
new_episode = True
while not game.is_episode_finished():
    s = game.get_state()
    if new_episode:
        print(s.game_variables)
        new_episode = False
    game.advance_action()
    a = game.get_last_action()
    r = game.get_last_reward()
game.close()

In [ ]:
env = VizDoomEnv('vizdoom_data/train_SPTM.cfg', 100, [1600, 1500], grayscale=False, map_id=5)

In [ ]:
for state, position in env.sample_states:
    print(position)

In [ ]:
model = resnet.ResnetBuilder.build_siamese_resnet_18((6, 120, 160), 2)

In [ ]:
model.load_weights('../../sptm/experiments/0103_R/models/model_weights.h5')

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
def get_features(state, norm=True):
    model_input = np.expand_dims(np.transpose(state, [1, 2, 0]), 0)
    features = model.layers[3].predict(model_input).squeeze()
    if norm:
        features = features / np.linalg.norm(features, ord=2)
    return features
def get_edge_prediction(A, B):
    concat = np.concatenate((A, B), axis=0)
    model_input = np.expand_dims(np.transpose(concat, [1, 2, 0]), 0)
    return model.predict(model_input)[:, 1]
def get_valid(data, m=3):
    return abs(data - np.mean(data)) < m * np.std(data)

In [ ]:
start_state, start_position = env.get_obs_at((1200, 1000))
start_state_features = get_features(start_state)

In [ ]:
sample_x = np.random.randint(env.min_x, env.max_x, 100)
sample_y = np.random.randint(env.min_y, env.max_y, 100)
samples = []
sample_states = []

In [ ]:
for x, y in zip(sample_x, sample_y):
    state, position = env.get_obs_at((x, y))
    features = get_features(state)
    similarity = np.dot(features, start_state_features)
    prediction = get_edge_prediction(start_state, state).item()
    samples.append((*position, similarity, prediction))
    sample_states.append(state)
samples = np.array(samples)

In [ ]:
env.plot_topdown()
plt.scatter(samples[:, 0], samples[:, 1], c=samples[:, 3], s=81)
plt.scatter(*start_position, marker='D')
plt.colorbar()